In [5]:
from haystack.utils import clean_wiki_text, convert_files_to_docs, fetch_archive_from_http, print_answers
from haystack.nodes import FARMReader
from haystack.nodes import TextConverter, PDFToTextConverter, PreProcessor
import os

import warnings
warnings.filterwarnings("ignore")

In [2]:
from haystack.document_stores import InMemoryDocumentStore,SQLDocumentStore

document_store = InMemoryDocumentStore()

# document_store = SQLDocumentStore(url="sqlite:///qa.db")

INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0


In [6]:
# filename = "Employee Manual_India.pdf"
# converter = PDFToTextConverter(remove_numeric_tables=True, valid_languages=["en"])
# doc_pdf = converter.convert(file_path=str("E:\\AA_TE_SEM_VI\\persistant\\learning material\\contextual search\\data\\" + filename), meta=None)[0]
# doc_pdf
data_dir = os.getcwd() + "\\data\\"

all_docs = convert_files_to_docs(dir_path=data_dir)

INFO - haystack.utils.preprocessing -  Converting E:\AA_TE_SEM_VI\persistant\learning material\contextual search\data\24_Acceptable Use Policy.pdf
INFO - haystack.utils.preprocessing -  Converting E:\AA_TE_SEM_VI\persistant\learning material\contextual search\data\Employee Manual_India.pdf


In [10]:
all_docs[0]

<Document: {'content': 'Persistent India Bhageerath, 402 Senapati Bapat Road, Pune 411 016 • +91 (20) 6703 0000\nAcceptable Use Policy\nEnterprise Information Systems\nImportant: The printed copy of this document is not under control\x0cPSL-ISC-POL-016 Persistent Confidential Page | 2\nDocument Control\nDocument\nNo.\nDocument\nName\nExecutive Summary Revision\nNo.\nEffective\nfrom Date\nClassification\nPSL-ISC-\nPOL-016\nAcceptable\nUse Policy\nThis policy establishes\nthe acceptable handling\nand usage of Persistent\ninformation systems by\nPersistent users.\n6.4 02 May 2022 Internal Use Only\nAuthorizations\nOwner Custodian Distribution List\nEIS EIS InfoSec & Compliance DU Persistent Systems and\nSubsidiaries\x0cPSL-ISC-POL-016 Persistent Confidential Page | 3\nTable of Contents\n1. Preamble.........................................................................................................................................4\n2. Terminology and Abbreviations......................

In [11]:
preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True, #false hota
    split_by="word",
    split_length=100,
    split_respect_sentence_boundary=True,
)

docs = preprocessor.process(all_docs)
print(f"\n n_files_input: {len(all_docs)}\n n_docs_output: {len(docs)}")

document_store.write_documents(docs)

100%|████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.04docs/s]


 n_files_input: 2
 n_docs_output: 995


In [12]:
len(docs)

995

In [59]:
from haystack.nodes import BM25Retriever,TfidfRetriever
from haystack.pipelines import ExtractiveQAPipeline
from haystack.nodes import FARMReader, TransformersReader
from haystack.nodes import EmbeddingRetriever

In [17]:
retriever = TfidfRetriever(document_store=document_store)
# retriever = BM25Retriever(document_store=document_store) #worked with elasticsearch as document store on colab

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2")

pipe = ExtractiveQAPipeline(reader, retriever)

INFO - haystack.nodes.retriever.sparse -  Found 995 candidate paragraphs from 995 docs in DB
INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find deepset/roberta-base-squad2 locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...
INFO - haystack.modeling.model.language_model -  Loaded deepset/roberta-base-squad2
INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
INFO - haystack.modeling.infer -  Got ya 7 parallel workers to do inference ...
INFO - haystack.modeling.infer -   0     0     0     0     0     0     0  
INFO - haystack.modeling.infer -  /w\   /w\   /w\   /w\   /w\   /w\   /w\ 
INFO - haystack.modeling.infer -  /'\   / 

In [56]:
# prediction = pipe.run(query="What is the frequency of project party?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}})

prediction = pipe.run(query="What is the budget for project party?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}})

Inferencing Samples: 100%|███████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.18 Batches/s]


In [57]:
from pprint import pprint
# print(len(prediction))
pprint(prediction)


# {'answer': 'Once in six months', 
#  'type': 'extractive',
#  'score': 0.5211937427520752, 
#  'context': 'es are held outside normal working hours. 2. Budget:\n• Frequency: Once in six months (i.e. 2 times in year)\n• Amount: Rs. 750/- per head per duration\n',
#  'offsets_in_document': [{'start': 470, 'end': 488}],
#  'offsets_in_context': [{'start': 66, 'end': 84}],
#  'document_id': '6d7afbcd7156492026b208c99c427021', 
#  'meta': {'name': 'Employee Manual_India.pdf', '_split_id': 236}
# }

{'answers': [<Answer {'answer': 'Rs. 750/- per head per duration', 'type': 'extractive', 'score': 0.615727037191391, 'context': 'quency: Once in six months (i.e. 2 times in year)\n• Amount: Rs. 750/- per head per duration\nThe Project Party will be held for a Project/Function as a', 'offsets_in_document': [{'start': 522, 'end': 553}], 'offsets_in_context': [{'start': 60, 'end': 91}], 'document_id': '6d7afbcd7156492026b208c99c427021', 'meta': {'name': 'Employee Manual_India.pdf', '_split_id': 236}}>,
             <Answer {'answer': 'higher cost', 'type': 'extractive', 'score': 0.2474047765135765, 'context': 'ny. • If the handset is required for project purpose which would need higher cost that\nspecified above, an employee must raise HSPS using the appropri', 'offsets_in_document': [{'start': 154, 'end': 165}], 'offsets_in_context': [{'start': 70, 'end': 81}], 'document_id': '58392e11d63c348e31c300dcccc25425', 'meta': {'name': 'Employee Manual_India.pdf', '_split_id': 204}}>,
          

In [58]:
print_answers(prediction, details="medium")


Query: What is the budget for project party?
Answers:
[   {   'answer': 'Rs. 750/- per head per duration',
        'context': 'quency: Once in six months (i.e. 2 times in year)\n'
                   '• Amount: Rs. 750/- per head per duration\n'
                   'The Project Party will be held for a Project/Function as a',
        'score': 0.615727037191391},
    {   'answer': 'higher cost',
        'context': 'ny. • If the handset is required for project purpose which '
                   'would need higher cost that\n'
                   'specified above, an employee must raise HSPS using the '
                   'appropri',
        'score': 0.2474047765135765},
    {   'answer': 'Q1 unused budget',
        'context': 'Ex: Q1 unused budget will be added to Q2 budget. But\n'
                   'unused budget at the end of Q2 will lapse. • E-Certificate '
                   'is released to the awardee after the a',
        'score': 0.20571652054786682},
    {   'answer': '3. The budg

In [60]:
# prediction

In [61]:
retriever2 = EmbeddingRetriever(
    document_store=document_store,
    embedding_model="sentence-transformers/multi-qa-mpnet-base-dot-v1",
    model_format="sentence_transformers"
)

document_store.update_embeddings(retriever2)

reader2 = FARMReader(model_name_or_path="deepset/roberta-base-squad2")

pipe2 = ExtractiveQAPipeline(reader2, retriever2)

prediction2 = pipe2.run(query="What is the budget for project party?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}})

# print(len(prediction))
pprint(prediction2)
#~ 7-8min to train

INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
INFO - haystack.nodes.retriever.dense -  Init retriever using embeddings of model sentence-transformers/multi-qa-mpnet-base-dot-v1


Downloading:   0%|          | 0.00/737 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.65k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

Documents Processed: 10000 docs [06:28, 25.76 docs/s]                                                                
INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find deepset/roberta-base-squad2 locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...
INFO - haystack.modeling.model.language_model -  Loaded deepset/roberta-base-squad2
INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
INFO - haystack.modeling.infer -  Got ya 7 parallel workers to do inference ...
INFO - haystack.modeling.infer -   0     0     0     0     0     0     0  
INFO - haystack.modeling.infer -  /w\   /w\   /w\   /w\   /w\   /w\   /w\ 
INFO - haystack.m

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Inferencing Samples: 100%|███████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.27 Batches/s]


{'answers': [<Answer {'answer': 'Rs. 750/- per head per duration', 'type': 'extractive', 'score': 0.615727037191391, 'context': 'quency: Once in six months (i.e. 2 times in year)\n• Amount: Rs. 750/- per head per duration\nThe Project Party will be held for a Project/Function as a', 'offsets_in_document': [{'start': 522, 'end': 553}], 'offsets_in_context': [{'start': 60, 'end': 91}], 'document_id': '6d7afbcd7156492026b208c99c427021', 'meta': {'name': 'Employee Manual_India.pdf', '_split_id': 236}}>,
             <Answer {'answer': '100', 'type': 'extractive', 'score': 0.15544985234737396, 'context': 'reimbursement of the health insurance plan will be borne by\nthe employee. 100\nProject Party\nObjective To promote team spirit within project/dept./team', 'offsets_in_document': [{'start': 102, 'end': 105}], 'offsets_in_context': [{'start': 74, 'end': 77}], 'document_id': '73c61acae1650d457362e110cd88400c', 'meta': {'name': 'Employee Manual_India.pdf', '_split_id': 235}}>,
             <A

In [62]:
print_answers(prediction2, details="medium")


Query: What is the budget for project party?
Answers:
[   {   'answer': 'Rs. 750/- per head per duration',
        'context': 'quency: Once in six months (i.e. 2 times in year)\n'
                   '• Amount: Rs. 750/- per head per duration\n'
                   'The Project Party will be held for a Project/Function as a',
        'score': 0.615727037191391},
    {   'answer': '100',
        'context': 'reimbursement of the health insurance plan will be borne '
                   'by\n'
                   'the employee. 100\n'
                   'Project Party\n'
                   'Objective To promote team spirit within project/dept./team',
        'score': 0.15544985234737396},
    {   'answer': '3. The budget allocated is for the specified duration',
        'context': 'U or project are eligible for the project party. 3. The '
                   'budget allocated is for the specified duration. the unused '
                   'budget for H1 (Apr to\n'
                   'Sept) wil

In [64]:
import os
import time
from subprocess import Popen, PIPE, STDOUT
from haystack.document_stores import ElasticsearchDocumentStore
from haystack.utils import launch_es
curr_dir = os.getcwd()

launch_es()
es_server = Popen([curr_dir + "\\elasticsearch-7.9.2\\bin\\elasticsearch.bat"], stdout=PIPE, stderr=STDOUT)

print("Waiting for ES to start...")
time.sleep(30) # Waiting until ES has started

# JubJObatm9NMq3cJLOa4
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

WARNING - haystack.utils.doc_store -  Tried to start Elasticsearch through Docker but this failed. It is likely that there is already an existing Elasticsearch instance running. 


Waiting for ES to start...


WARNING - elasticsearch -  HEAD http://localhost:9200/ [status:N/A request:4.527s]
Traceback (most recent call last):
  File "C:\Users\Admin\anaconda3\envs\cxt_srch\lib\site-packages\urllib3\connection.py", line 175, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw
  File "C:\Users\Admin\anaconda3\envs\cxt_srch\lib\site-packages\urllib3\util\connection.py", line 95, in create_connection
    raise err
  File "C:\Users\Admin\anaconda3\envs\cxt_srch\lib\site-packages\urllib3\util\connection.py", line 85, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Admin\anaconda3\envs\cxt_srch\lib\site-packages\elasticsearch\connection\http_urllib3.py", line 246, in perform_request
    method, url, body, retries=Retry(False), headers=request_headers, *

WARNING - elasticsearch -  HEAD http://localhost:9200/ [status:N/A request:4.114s]
Traceback (most recent call last):
  File "C:\Users\Admin\anaconda3\envs\cxt_srch\lib\site-packages\urllib3\connection.py", line 175, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw
  File "C:\Users\Admin\anaconda3\envs\cxt_srch\lib\site-packages\urllib3\util\connection.py", line 95, in create_connection
    raise err
  File "C:\Users\Admin\anaconda3\envs\cxt_srch\lib\site-packages\urllib3\util\connection.py", line 85, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Admin\anaconda3\envs\cxt_srch\lib\site-packages\elasticsearch\connection\http_urllib3.py", line 246, in perform_request
    method, url, body, retries=Retry(False), headers=request_headers, *

ConnectionError: Initial connection to Elasticsearch failed. Make sure you run an Elasticsearch instance at `[{'host': 'localhost', 'port': 9200}]` and that it has finished the initial ramp up (can take > 30s).